In [1]:
from google.colab import drive
drive.mount('/content/drive') #1397

Mounted at /content/drive


# Imports

In [2]:
! pip install datasets transformers

     |████████████████████████████████| 264 kB 8.8 MB/s 
     |████████████████████████████████| 2.8 MB 45.9 MB/s 
     |████████████████████████████████| 50 kB 5.5 MB/s 
     |████████████████████████████████| 243 kB 49.3 MB/s 
     |████████████████████████████████| 119 kB 43.3 MB/s 
     |████████████████████████████████| 636 kB 51.3 MB/s 
     |████████████████████████████████| 895 kB 41.0 MB/s 
     |████████████████████████████████| 3.3 MB 37.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
VERSION = "1.8.1"
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  85266      0 --:--:-- --:--:-- --:--:-- 85266
Updating... This may take around 2 minutes.
Found existing installation: torch 1.9.0+cu102
Uninstalling torch-1.9.0+cu102:
  Successfully uninstalled torch-1.9.0+cu102
Found existing installation: torchvision 0.10.0+cu102
Uninstalling torchvision-0.10.0+cu102:
  Successfully uninstalled torchvision-0.10.0+cu102
Copying gs://tpu-pytorch/wheels/torch-1.8.1-cp37-cp37m-linux_x86_64.whl...
\ [1 files][126.5 MiB/126.5 MiB]                                                
Operation completed over 1 objects/126.5 MiB.                                    
Copying gs://tpu-pytorch/wheels/torch_xla-1.8.1-cp37-cp37m-linux_x86_64.whl...
\ [1 files][138.2 MiB/138.2 MiB]                                                
Operation completed over 1 objects/138.2 MiB.       

In [3]:
import pandas as pd
import numpy as np
import transformers
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer, default_data_collator, AutoTokenizer

import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp

print(transformers.__version__)

4.10.2


# Data Preprocessing

In [4]:
from datasets import load_dataset, load_metric

data = load_dataset("squad_v2")
metric = load_metric("squad_v2")

Downloading:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.55M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/801k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


Downloading:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

In [5]:
# for i in data:
#   data[i] = data[i].select(range(500))

data

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [6]:
model_name = "tau/splinter-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/145 [00:00<?, ?B/s]

Tokenization

In [7]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

pad_on_right = tokenizer.padding_side == "right"

def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [8]:
processed_data = data.map(prepare_train_features, batched=True, remove_columns=data["train"].column_names)

processed_data

  0%|          | 0/131 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'end_positions', 'input_ids', 'start_positions', 'token_type_ids'],
        num_rows: 132131
    })
    validation: Dataset({
        features: ['attention_mask', 'end_positions', 'input_ids', 'start_positions', 'token_type_ids'],
        num_rows: 12208
    })
})

# Fine Tuning

In [10]:
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

model.train()
WRAPPED_MODEL = xmp.MpModelWrapper(model)

Downloading:   0%|          | 0.00/431M [00:00<?, ?B/s]

Some weights of SplinterForQuestionAnswering were not initialized from the model checkpoint at tau/splinter-base and are newly initialized: ['splinter_qass.start_transform.LayerNorm.bias', 'splinter_qass.start_transform.LayerNorm.weight', 'splinter_qass.start_classifier.weight', 'splinter_qass.query_start_transform.LayerNorm.bias', 'splinter_qass.end_transform.LayerNorm.weight', 'splinter_qass.start_transform.dense.bias', 'splinter_qass.end_transform.LayerNorm.bias', 'splinter_qass.query_start_transform.dense.weight', 'splinter_qass.end_classifier.weight', 'splinter_qass.query_end_transform.dense.weight', 'splinter_qass.end_transform.dense.weight', 'splinter_qass.query_start_transform.LayerNorm.weight', 'splinter_qass.query_start_transform.dense.bias', 'splinter_qass.query_end_transform.dense.bias', 'splinter_qass.query_end_transform.LayerNorm.weight', 'splinter_qass.start_transform.dense.weight', 'splinter_qass.query_end_transform.LayerNorm.bias', 'splinter_qass.end_transform.dense.bi

# TPU

In [11]:
def train_loop(model, batch_size=32):
    """
    This contains everything that must be done to train our models
    """
    print("Training... ", end="")

    training_args = TrainingArguments(
        "Q&A",
        evaluation_strategy = "epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=3,
        tpu_num_cores=8
    )

    trainer = Trainer(
        model,
        training_args,
        train_dataset=processed_data["train"],
        eval_dataset=processed_data["validation"]
    )

    trainer.place_model_on_device = False
    trainer.train()

    trainer.save_model('/content/drive/MyDrive/Q&A/Splinter')

In [12]:
def _mp_fn(index):

    device = xm.xla_device()

    model = WRAPPED_MODEL.to(device)

    train_loop(model)

xmp.spawn(_mp_fn, start_method="fork")

***** Running training *****
  Num examples = 132131
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 1551


Training... 

Epoch,Training Loss,Validation Loss
1,1.676500,0.868286
2,0.856700,0.811294
3,0.736400,0.884142


Training... 

Epoch,Training Loss,Validation Loss
1,1.985800,0.868286
2,0.830100,0.811294
3,0.729200,0.884142


...

Training... 

Epoch,Training Loss,Validation Loss
1,1.668500,0.868286
2,0.862200,0.811294
3,0.732000,0.884142


Training... 

Epoch,Training Loss,Validation Loss
1,1.665900,0.868286
2,1.492200,0.811294
3,1.666900,0.884142


Saving model checkpoint to Q&A/checkpoint-500
Configuration saved in Q&A/checkpoint-500/config.json
Model weights saved in Q&A/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 12208
  Batch size = 32
Saving model checkpoint to Q&A/checkpoint-1000
Configuration saved in Q&A/checkpoint-1000/config.json
Model weights saved in Q&A/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 12208
  Batch size = 32
Saving model checkpoint to Q&A/checkpoint-1500
Configuration saved in Q&A/checkpoint-1500/config.json
Model weights saved in Q&A/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 12208
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/Q&A/Splinter
Configuration saved in /content/drive/MyDrive/Q&A/Splinter/config.json
Model weights saved in /content/drive/MyDrive/Q&A/Splinter/pytorch_model.bin


# Evaluation

In [9]:
model = AutoModelForQuestionAnswering.from_pretrained("/content/drive/MyDrive/Q&A/Splinter") # 3 Epoch

training_args = TrainingArguments(
    "Q&A",
    per_device_eval_batch_size=32
)

trainer = Trainer(
    model,
    training_args
)

In [10]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [11]:
validation_data = data["validation"].map(prepare_validation_features, batched=True, remove_columns=data["validation"].column_names)

validation_data

  0%|          | 0/12 [00:00<?, ?ba/s]

Dataset({
    features: ['attention_mask', 'example_id', 'input_ids', 'offset_mapping', 'token_type_ids'],
    num_rows: 12208
})

In [12]:
raw_pred = trainer.predict(validation_data)

raw_pred

The following columns in the test set  don't have a corresponding argument in `SplinterForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id.
***** Running Prediction *****
  Num examples = 12208
  Batch size = 32


PredictionOutput(predictions=(array([[   30.050335,    22.228977,    20.075397, ..., -9977.262   ,
        -9977.602   , -9977.457   ],
       [   35.297153,    26.425823,    22.849775, ..., -9976.034   ,
        -9975.951   , -9975.941   ],
       [   33.860485,    23.248848,    21.752018, ..., -9980.474   ,
        -9980.353   , -9980.1045  ],
       ...,
       [   42.92713 ,    28.846228,    27.143162, ..., -9976.883   ,
        -9976.995   , -9976.279   ],
       [   42.915062,    30.47486 ,    30.961447, ..., -9973.421   ,
        -9973.479   , -9972.929   ],
       [   42.31097 ,    27.845592,    30.005539, ..., -9973.3955  ,
        -9973.853   , -9973.879   ]], dtype=float32), array([[   30.510662,    19.906202,    16.640612, ..., -9980.318   ,
        -9980.753   , -9980.479   ],
       [   32.39646 ,    22.088825,    16.324463, ..., -9981.486   ,
        -9981.513   , -9981.591   ],
       [   33.580837,    21.192324,    18.4598  , ..., -9983.291   ,
        -9983.526   , -9

In [18]:
import collections
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions

In [19]:
final_predictions = postprocess_qa_predictions(data["validation"], validation_data, raw_pred.predictions)

result = pd.DataFrame({'id': data['validation']['id'],
                       'question': data['validation']['question']})

for i, j in final_predictions.items():
  result.loc[result.id == i, 'answer'] = j

result

Post-processing 11873 example predictions split into 12208 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

,id,question,answer
0,56ddde6b9a695914005b9628,In what country is Normandy located?,France
1,56ddde6b9a695914005b9629,When were the Normans in Normandy?,10th and 11th centuries
2,56ddde6b9a695914005b962a,From which countries did the Norse originate?,"Denmark, Iceland and Norway"
3,56ddde6b9a695914005b962b,Who was the Norse leader?,Rollo
4,56ddde6b9a695914005b962c,What century did the Normans first gain their ...,10th
...,...,...,...
11868,5737aafd1c456719005744ff,What is the seldom used force unit equal to on...,
11869,5ad28ad0d7d075001a4299cc,What does not have a metric counterpart?,
11870,5ad28ad0d7d075001a4299cd,What is the force exerted by standard gravity ...,
11871,5ad28ad0d7d075001a4299ce,What force leads to a commonly used unit of mass?,


In [23]:
result[result.answer != ""]

,id,question,answer
0,56ddde6b9a695914005b9628,In what country is Normandy located?,France
1,56ddde6b9a695914005b9629,When were the Normans in Normandy?,10th and 11th centuries
2,56ddde6b9a695914005b962a,From which countries did the Norse originate?,"Denmark, Iceland and Norway"
3,56ddde6b9a695914005b962b,Who was the Norse leader?,Rollo
4,56ddde6b9a695914005b962c,What century did the Normans first gain their ...,10th
...,...,...,...
11856,5737a9afc3c5551400e51f62,What changes macroscopic closed system energies?,nonconservative forces
11858,5737a9afc3c5551400e51f64,What is the law of thermodynamics associated w...,Second law of thermodynamics
11859,5737a9afc3c5551400e51f65,What makes energy changes in a closed system?,entropy increases
11865,5737aafd1c456719005744fc,What is the kilogram-force sometimes reffered ...,kilopond


In [20]:
formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in data["validation"]]

metric.compute(predictions=formatted_predictions, references=references)

{'HasAns_exact': 76.889338731444,
 'HasAns_f1': 83.13920258653401,
 'HasAns_total': 5928,
 'NoAns_exact': 80.0,
 'NoAns_f1': 80.0,
 'NoAns_total': 5945,
 'best_exact': 78.4468963193801,
 'best_exact_thresh': 0.0,
 'best_f1': 81.56735390659256,
 'best_f1_thresh': 0.0,
 'exact': 78.4468963193801,
 'f1': 81.56735390659259,
 'total': 11873}